<a href="https://colab.research.google.com/github/osu-mp/ai-539-nlp-group/blob/main/PruningNLPProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Things Needed to Run Code 

Install transformers (From HuggingFace)

Install all needed libraries here.

Run these lines everytime you open in Basic Colab.


-Datasets are needed for training
-Sacremoses is needed for SARI

In [19]:
!pip install -q transformers
!pip install datasets
!pip install -q sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
!nvidia-smi

Tue Mar 14 20:22:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   29C    P0    51W / 400W |  24245MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

**Imports**

Import all necessary packages here.

In [21]:
%load_ext autoreload
%autoreload 2
import argparse
import glob
import logging
import os
import pickle
import random
import re
import shutil
from typing import Dict, List, Tuple
from urllib.request import urlretrieve
import bz2
import gzip
import os
from pathlib import Path
import sys
import tarfile
import tempfile
import time
from urllib.request import urlretrieve
import zipfile
from datasets import load_dataset
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
from torch.utils.data.distributed import DistributedSampler
import torch.nn.utils.prune as prune
from tqdm import tqdm, trange
import torch.nn.utils.prune as prune
import pdb  
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, AutoTokenizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
cuda


In [22]:
GOOGLE_DRIVE_PATH = os.path.join('drive', 'My Drive')
import sys
from google.colab import drive
sys.path.append(GOOGLE_DRIVE_PATH)
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [25]:
import copy
def get_weight_parameters(layer):
    '''
    Get all parameters/modules identified as 'weight'
    '''
    weight_parameters = []
    if len(list(layer.children())) > 0:
        for child in layer.children():
            for param in child.named_parameters():
                if 'weight' == param[0]:
                    weight_parameters.append((child, param[0]))
            weight_parameters.extend(get_weight_parameters(child))
    
    
    return weight_parameters


def prune_weight_parameters(model, prune_amount):
    '''
    Global pruning
    '''
    params_to_prune = get_weight_parameters(model)

    prune.global_unstructured(
        params_to_prune, 
        pruning_method=prune.L1Unstructured, 
        amount=prune_amount,
    )

    for module, name in params_to_prune:
        try:
            prune.remove(module, name)
            print(module)
        except Exception as e:
            print(e)
    return model

def get_pruned_models(model, sparsity):
    model_to_prune = copy.deepcopy(model)
    pruned_model = prune_weight_parameters(model_to_prune, sparsity)
    return pruned_model

# Load and Train BART

Here we load the model. We have options between "bart-base" or "bart-large". 

In [26]:
# Load the model from Hugging Face
from transformers import BartTokenizer, BartForConditionalGeneration, AdamW, BartConfig
import torch.nn.utils.prune as prune
model_name = 'facebook/bart-base'
bart_tokenizer = BartTokenizer.from_pretrained(model_name, add_prefix_space=True)
bart_model_huggingface = BartForConditionalGeneration.from_pretrained(model_name)
bart_model_huggingface = get_pruned_models(bart_model_huggingface, 20)

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/merges.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,

Embedding(50265, 768, padding_idx=1)
Embedding(50265, 768, padding_idx=1)
BartLearnedPositionalEmbedding(1026, 768)
Linear(in_features=768, out_features=768, bias=True)
Linear(in_features=768, out_features=768, bias=True)
Linear(in_features=768, out_features=768, bias=True)
Linear(in_features=768, out_features=768, bias=True)
LayerNorm((768,), eps=1e-05, elementwise_affine=True)
Linear(in_features=768, out_features=3072, bias=True)
Linear(in_features=3072, out_features=768, bias=True)
LayerNorm((768,), eps=1e-05, elementwise_affine=True)
Linear(in_features=768, out_features=768, bias=True)
Linear(in_features=768, out_features=768, bias=True)
Linear(in_features=768, out_features=768, bias=True)
Linear(in_features=768, out_features=768, bias=True)
LayerNorm((768,), eps=1e-05, elementwise_affine=True)
Linear(in_features=768, out_features=3072, bias=True)
Linear(in_features=3072, out_features=768, bias=True)
LayerNorm((768,), eps=1e-05, elementwise_affine=True)
Linear(in_features=768, out_

In [7]:
cnn_dailymail_dataset = load_dataset("cnn_dailymail",'3.0.0')

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

Dataset cnn_dailymail downloaded and prepared to /root/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
print(cnn_dailymail_dataset["train"][0])
print(cnn_dailymail_dataset["validation"][0])
print(cnn_dailymail_dataset["test"][0])

{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office char

In [9]:
prefix = "summarize: " # we define a prefix and attach it to every article in training data
def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["article"]]
    model_inputs = bart_tokenizer(inputs, max_length=1024, truncation=True)

    with bart_tokenizer.as_target_tokenizer():
        labels = bart_tokenizer(examples["highlights"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs
  
tokenized_cnn_dailymail_train = cnn_dailymail_dataset["train"].map(preprocess_function, batched=True) # we apply the function we defined to the training dataset
tokenized_cnn_dailymail_validation = cnn_dailymail_dataset["validation"].map(preprocess_function, batched=True) #for validation data
tokenized_cnn_dailymail_test = cnn_dailymail_dataset["test"].map(preprocess_function, batched=True) # for test data


Map:   0%|          | 0/287113 [00:00<?, ? examples/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:3581: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/13368 [00:00<?, ? examples/s]

Map:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [10]:
train_dataloader = DataLoader(tokenized_cnn_dailymail_train, batch_size=32, num_workers=4, collate_fn=lambda x: x)
valid_dataloader = DataLoader(tokenized_cnn_dailymail_validation, batch_size=32, num_workers=4, collate_fn=lambda x: x)
test_dataloader = DataLoader(tokenized_cnn_dailymail_test, batch_size=32, num_workers=4, collate_fn=lambda x: x)
i = 0
for batch in train_dataloader:
  print(batch)
  i = i + 1
  if i == 3:
    break

[{'article': 'LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won\'t cast a spell on him. Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties. "I don\'t plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month. "I don\'t think I\'ll be particularly extravagant. "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs." At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie on the UK box office cha

In [11]:
data_collator = DataCollatorForSeq2Seq(tokenizer=bart_tokenizer, model=model_name)
type(tokenized_cnn_dailymail_train)


datasets.arrow_dataset.Dataset

In [12]:
modules = [module for module in bart_model_huggingface.modules()]
params = [param.shape for param in bart_model_huggingface.parameters()]


print(modules[0])
'''
total_params=0
for i in range(1,len(modules)):
   j = 2*i
   param = (params[j-2][1]*params[j-2][0])+params[j-1][0]
   total_params += param
   print("Layer",i,"->\t",end="")
   print("Weights:", params[j-2][0],"x",params[j-2][1],
         "\tBias: ",params[j-1][0], "\tParameters: ", param)
print("\nTotal Params: ", total_params)
'''


BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

'\ntotal_params=0\nfor i in range(1,len(modules)):\n   j = 2*i\n   param = (params[j-2][1]*params[j-2][0])+params[j-1][0]\n   total_params += param\n   print("Layer",i,"->\t",end="")\n   print("Weights:", params[j-2][0],"x",params[j-2][1],\n         "\tBias: ",params[j-1][0], "\tParameters: ", param)\nprint("\nTotal Params: ", total_params)\n'

One Shot Magnitude pruning logic

In [13]:
# Algorithm
# Identify parameters to prune
# Prune the model
# Pass the model to seq2seqtrainer
# Run the evaluation metric
#Test

Iterative Magnitude Pruning

In [14]:
#torch.cuda.empty_cache()
#del train_dataloader, valid_dataloader, test_dataloader
#del cnn_dailymail_dataset
#del trainer
#!nvidia-smi



In [15]:
#Test
bart_model_huggingface = bart_model_huggingface.to(device)

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=5,
)

trainer = Seq2SeqTrainer(
    model=bart_model_huggingface,
    args=training_args,
    train_dataset=tokenized_cnn_dailymail_train,
    eval_dataset=tokenized_cnn_dailymail_validation,
    tokenizer=bart_tokenizer,
    data_collator=data_collator,
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: article, highlights, id. If article, highlights, id are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 287113
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 89725
  Number of trainable parameters = 139420416


Epoch,Training Loss,Validation Loss
1,1.854900,1.647890
2,1.754800,1.608496
3,1.689300,1.582782
4,1.640700,1.567843


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Configuration saved in ./results/checkpoint-500/generation_config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1000
Configuration saved in ./results/checkpoint-1000/config.json
Configuration saved in ./results/checkpoint-1000/generation_config.json
Model weights saved in ./results/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results/checkpoint-1500
Configuration saved in ./results/checkpoint-1500/config.json
Configuration saved in ./results/checkpoint-1500/generation_config.jso

Epoch,Training Loss,Validation Loss
1,1.854900,1.647890
2,1.754800,1.608496
3,1.689300,1.582782
4,1.640700,1.567843
5,1.608400,1.565175


Saving model checkpoint to ./results/checkpoint-88500
Configuration saved in ./results/checkpoint-88500/config.json
Configuration saved in ./results/checkpoint-88500/generation_config.json
Model weights saved in ./results/checkpoint-88500/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-88500/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-88500/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-87000] due to args.save_total_limit
Saving model checkpoint to ./results/checkpoint-89000
Configuration saved in ./results/checkpoint-89000/config.json
Configuration saved in ./results/checkpoint-89000/generation_config.json
Model weights saved in ./results/checkpoint-89000/pytorch_model.bin
tokenizer config file saved in ./results/checkpoint-89000/tokenizer_config.json
Special tokens file saved in ./results/checkpoint-89000/special_tokens_map.json
Deleting older checkpoint [results/checkpoint-87500] due to args.save_total_limit
Sa

TrainOutput(global_step=89725, training_loss=1.7340125294002726, metrics={'train_runtime': 53229.8811, 'train_samples_per_second': 26.969, 'train_steps_per_second': 1.686, 'total_flos': 8.751740595358925e+17, 'train_loss': 1.7340125294002726, 'epoch': 5.0})

In [16]:
trainer.save_model('/content/drive/MyDrive/AI539MUSSLT/oneshot-80-bart-base.pt')
trainer.evaluate(cnn_dailymail_dataset["test"])
# TODO
# train loop
# eval loop
# save model 





Saving model checkpoint to /content/drive/MyDrive/AI539MUSSLT/oneshot-80-bart-base.pt
Configuration saved in /content/drive/MyDrive/AI539MUSSLT/oneshot-80-bart-base.pt/config.json
Configuration saved in /content/drive/MyDrive/AI539MUSSLT/oneshot-80-bart-base.pt/generation_config.json
Model weights saved in /content/drive/MyDrive/AI539MUSSLT/oneshot-80-bart-base.pt/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/AI539MUSSLT/oneshot-80-bart-base.pt/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/AI539MUSSLT/oneshot-80-bart-base.pt/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BartForConditionalGeneration.forward` and have been ignored: article, highlights, id. If article, highlights, id are not expected by `BartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 0
  Batch size = 16


{'eval_runtime': 0.0054,
 'eval_samples_per_second': 0.0,
 'eval_steps_per_second': 0.0,
 'epoch': 5.0}

TODO: modify BERT pruning to handle BART

# Evaluation Step with the SARI metric

Our dataset uses a subset of the easse repo. Copies have been made in the AI539MUSSLT folder. Load them here:

In [17]:
from drive.MyDrive.AI539MUSSLT import sari,easse_prep

In [18]:
#model_path = '/content/drive/MyDrive/AI539MUSSLT/bart-base.pt'
#bart_model_fairseq = torch.load(model_path)
# Reference - https://github.com/facebookresearch/muss/blob/7a8d6f63a62f5ec8d340d7ce71058111adaf6b01/muss/mining/training.py#LL42C4-L43C61
#for key in ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight']:
#  bart_model_fairseq['model'][key] = bart_model_fairseq['model'][key][:50264, :]

#!tar -xzf '/content/drive/MyDrive/AI539MUSSLT/cnn_stories.tgz' --directory '/content/drive/MyDrive/AI539MUSSLT/'
#!tar -xzf '/content/drive/MyDrive/AI539MUSSLT/dailymail_stories.tgz' --directory '/content/drive/MyDrive/AI539MUSSLT/'
#sys.path.append('/content/drive/MyDrive/AI539MUSSLT/')
#!python /content/drive/MyDrive/AI539MUSSLT/make_datafiles.py '/content/drive/MyDrive/AI539MUSSLT/cnn/stories' '/content/drive/MyDrive/AI539MUSSLT/dailymail/stories'
#!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/encoder.json' -P  '/content/drive/MyDrive/AI539MUSSLT/'
#!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/vocab.bpe' -P  '/content/drive/MyDrive/AI539MUSSLT/'
#!wget -N 'https://dl.fbaipublicfiles.com/fairseq/gpt2_bpe/dict.txt' -P  '/content/drive/MyDrive/AI539MUSSLT/'